In [1]:
import jerel
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits

from marvin.tools import Maps

with fits.open('./data/galaxies_sorted.fits') as hdulist:
    plateifu_list = hdulist[1].data['plateifu']
    
with fits.open('./data/angles.fits') as hdulist:
    angle_list = hdulist[1].data['col2']

angle_list[0] = -160
angle_list[11] = 110
angle_list[10] = 125
angle_list[12] = -70
angle_list[13] = -140

import warnings 
warnings.filterwarnings("ignore")

[INFO]: No release version set. Setting default to DR15


In [2]:
t= jerel.stat_list(plateifu_list, angle_list)

In [3]:
t

num,ifu_id,d_angle,l_area,t_area,l_sum,t_sum,l_error,t_error,l_median,t_median,asy
int64,str11,int16,int64,int64,float64,float64,float64,float64,float64,float64,float64
0,8944-6101,-160,100,102,0.016043712484178376,0.016419162499116565,1.4408112556034058e-05,1.522339573267205e-05,0.015343508234838193,0.01302500187653026,0.3354955058284942
1,8934-3701,40,170,145,0.03019096780811817,0.012056494968548874,5.792688106066565e-06,3.748807303925197e-06,0.019002316838863692,0.006560421224111978,0.5955885442533122
2,8932-12701,-25,1329,1233,0.0034308540889914107,0.0020853444573131884,1.1622876013170373e-06,1.0692381096222191e-06,0.002908195900117213,0.001337174448615124,0.4392495575707111
3,9876-3702,-120,169,170,0.02096682845123189,0.014856238904006867,7.730849084997975e-06,6.213805426072259e-06,0.011822061279720615,0.011027006782692805,0.2505285530306671
4,8932-3701,-90,253,253,0.025766711865142882,0.017620925805628507,6.280180732350374e-06,5.527844193255498e-06,0.02473935962960708,0.015093825271254349,0.23247671977615336
5,8950-12705,-20,166,281,0.00803474184897874,0.0056553890977646785,1.2425266281668121e-05,6.7124264151337305e-06,0.007700803433307441,0.004832644073847047,0.31432917354556067
6,8950-1901,-130,71,72,0.03010673334928013,0.027227026968972168,2.397583419853782e-05,2.3095105439683038e-05,0.02992553965393619,0.024070772496815912,0.1720376613290293
7,8931-3703,10,163,175,0.0037294021189813427,0.003456261787815674,3.442987784616112e-06,3.151877788737329e-06,0.002072971482195873,0.0024541124202225313,0.18063296611386667
8,9876-3703,180,235,244,0.0023075485901656793,0.0019266249275786784,1.3429452390571923e-06,1.2619235139171251e-06,0.0010653570895550457,0.0009051067266470307,0.2554289202744723


In [4]:
t.write('./stat/stat_sfr.fits', format='fits')

In [ ]:
(l_median-t_median)/(l_median+t_median)